In [12]:
import tensorflow as tf
import numpy as np
import math
from censai import PhysicalModel, RIMSharedUnet
from censai.definitions import DTYPE
from censai.models import SharedUnetModel
from censai.data.lenses_tng import decode_train, decode_physical_model_info, preprocess
from censai.utils import nullwriter, rim_residual_plot as residual_plot, plot_to_image
import os, glob, time
from datetime import datetime
from tensorboard.plugins.hparams import api as hp
import random
    
    
RIM_HPARAMS = [
    "adam",
    "steps",
    "kappalog",
    "kappa_normalize",
    "kappa_init",
    "source_init"
]
UNET_MODEL_HPARAMS = [
    "filters",
    "filter_scaling",
    "kernel_size",
    "layers",
    "block_conv_layers",
    "strides",
    "bottleneck_kernel_size",
    "bottleneck_filters",
    "resampling_kernel_size",
    "input_kernel_size",
    "gru_kernel_size",
    "upsampling_interpolation",
    "batch_norm",
    "dropout_rate",
    "kernel_l2_amp",
    "bias_l2_amp",
    "kernel_l1_amp",
    "bias_l1_amp",
    "activation",
    "alpha",
    "initializer",
    "gru_architecture"
]

In [13]:
# from censai.models.layers import UnetDecodingLayer, UnetEncodingLayer
# # from censai.models.layers.conv_gru_component import ConvGRUBlock
# from censai.models.utils import get_activation
# from censai.definitions import DTYPE
# # from censai.models.layers.conv_gru import ConvGRU

# import tensorflow as tf
# from censai.definitions import DTYPE


# class ConvGRU(tf.keras.layers.Layer):
#     def __init__(self, filters=32, kernel_size=5, **kwargs):
#         super(ConvGRU, self).__init__(dtype=DTYPE, **kwargs)
#         self.filters = filters
#         self.kernel_size = kernel_size
    
#     def build(self, input_shape):
#         self.Wz = tf.keras.layers.Conv2D(
#             filters=self.filters,
#             kernel_size=self.kernel_size,
#             strides=(1, 1),
# #             activation='sigmoid',
#             padding='same',
# #             kernel_initializer=tf.keras.initializers.GlorotUniform()
#         )
#         self.Uz = tf.keras.layers.Conv2D(
#             filters=self.filters,
#             kernel_size=self.kernel_size,
#             strides=(1, 1),
# #             activation='sigmoid',
#             padding='same',
# #             kernel_initializer=tf.keras.initializers.GlorotUniform()
#         )
#         self.Bz = tf.Variable(tf.random.truncated_normal(shape=input_shape[1:], mean=0.0, stddev=0.5, dtype=DTYPE))
        
#         self.Wr = tf.keras.layers.Conv2D(
#             filters=self.filters,
#             kernel_size=self.kernel_size,
#             strides=(1, 1),
# #             activation='sigmoid',
#             padding='same',
# #             kernel_initializer=tf.keras.initializers.GlorotUniform()
#         )
#         self.Ur = tf.keras.layers.Conv2D(
#             filters=self.filters,
#             kernel_size=self.kernel_size,
#             strides=(1, 1),
# #             activation='sigmoid',
#             padding='same',
# #             kernel_initializer=tf.keras.initializers.GlorotUniform()
#         )
#         self.Br = tf.Variable(tf.random.truncated_normal(shape=input_shape[1:], mean=0.0, stddev=0.5, dtype=DTYPE))
        
#         self.Wh = tf.keras.layers.Conv2D(
#             filters=self.filters,
#             kernel_size=self.kernel_size,
#             strides=(1, 1),
# #             activation='sigmoid',
#             padding='same',
# #             kernel_initializer=tf.keras.initializers.GlorotUniform()
#         )
#         self.Uh = tf.keras.layers.Conv2D(
#             filters=self.filters,
#             kernel_size=self.kernel_size,
#             strides=(1, 1),
# #             activation='sigmoid',
#             padding='same',
# #             kernel_initializer=tf.keras.initializers.GlorotUniform()
#         )
#         self.Bh = tf.Variable(tf.random.truncated_normal(shape=input_shape[1:], mean=0.0, stddev=0.5, dtype=DTYPE))


#     def call(self, x, ht):
#         """
#         Compute the new state tensor h_{t+1}.
#         """
# #         stacked_input = tf.concat([features, ht], axis=3)
#         z = tf.nn.sigmoid(self.Wz(x) + self.Uz(ht) + self.Bz)  # Update gate vector
#         r = tf.nn.sigmoid(self.Wr(x) + self.Ur(ht) + self.Br)    # Reset gate vector
#         h_tilde = tf.nn.tanh(self.Wh(x) + self.Uh(r * ht) + self.Bh)  # candidate activation
#         new_state = (1 - z) * ht + z * h_tilde
# #         r_state = tf.multiply(r, ht)
# #         stacked_r_state = tf.concat([features, r_state], axis=3)
# #         tilde_h = self.candidate_activation_gate(stacked_r_state)  # candidate activation gate
# #         new_state = tf.multiply(z, ht) + tf.multiply(1 - z, tilde_h)
#         return new_state  # h_{t+1}



# class ConvGRUBlock(tf.keras.Model):
#     """
#     Abstraction for the recurrent block inside the RIM
#     """
#     def __init__(
#             self,
#             filters,
#             kernel_size=5,
#             activation=tf.keras.layers.LeakyReLU()
#     ):
#         gru_filters = filters
#         super(ConvGRUBlock, self).__init__()
#         self.conv1 = tf.keras.layers.Conv2D(
#             filters=filters,
#             kernel_size=kernel_size,
#             strides=1,
#             activation=activation,
#             padding='same')
#         self.gru1 = ConvGRU(gru_filters, kernel_size)
#         self.gru2 = ConvGRU(gru_filters, kernel_size)

#     def call(self, inputs, state):
#         ht_11, ht_12 = tf.split(state, 2, axis=3)
#         gru_1_out  = self.gru1(inputs, ht_11)
#         gru_1_outE = self.conv1(gru_1_out)
#         gru_2_out  = self.gru2(gru_1_outE, ht_12)
#         ht = tf.concat([gru_1_out, gru_2_out], axis=3)
#         xt = gru_2_out
#         return xt, ht


# class SharedUnetModel(tf.keras.Model):
#     def __init__(
#             self,
#             name="RIMUnetModel",
#             filters=32,
#             filter_scaling=1,
#             kernel_size=3,
#             layers=2,                        # before bottleneck
#             block_conv_layers=2,
#             strides=2,
#             bottleneck_kernel_size=None,     # use kernel_size as default
#             bottleneck_filters=None,
#             resampling_kernel_size=None,
#             input_kernel_size=11,
#             gru_kernel_size=None,
#             batch_norm=False,
#             upsampling_interpolation=False,  # use strided transposed convolution if false
#             kernel_l1_amp=0.,
#             bias_l1_amp=0.,
#             kernel_l2_amp=0.,
#             bias_l2_amp=0.,
#             activation="leaky_relu",
#             alpha=0.1,                       # for leaky relu
#             use_bias=True,
#             trainable=True,
#             initializer="glorot_uniform",
#     ):
#         super(SharedUnetModel, self).__init__(name=name)
#         self.trainable = trainable

#         common_params = {"padding": "same", "kernel_initializer": initializer,
#                          "data_format": "channels_last", "use_bias": use_bias,
#                          "kernel_regularizer": tf.keras.regularizers.L1L2(l1=kernel_l1_amp, l2=kernel_l2_amp)}
#         if use_bias:
#             common_params.update({"bias_regularizer": tf.keras.regularizers.L1L2(l1=bias_l1_amp, l2=bias_l2_amp)})

#         resampling_kernel_size = resampling_kernel_size if resampling_kernel_size is not None else kernel_size
#         bottleneck_kernel_size = bottleneck_kernel_size if bottleneck_kernel_size is not None else kernel_size
#         bottleneck_filters = bottleneck_filters if bottleneck_filters is not None else int(filter_scaling**(layers + 1) * filters)
#         gru_kernel_size = gru_kernel_size if gru_kernel_size is not None else kernel_size
#         activation = get_activation(activation, alpha=alpha)

#         self._num_layers = layers
#         self._strides = strides
#         self._init_filters = filters
#         self._filter_scaling = filter_scaling
#         self._bottleneck_filters = bottleneck_filters

#         self.encoding_layers = []
#         self.decoding_layers = []
#         self.gated_recurrent_blocks = []
#         for i in range(layers):
#             self.encoding_layers.append(
#                 UnetEncodingLayer(
#                     kernel_size=kernel_size,
#                     downsampling_kernel_size=resampling_kernel_size,
#                     filters=int(filter_scaling**(i) * filters),
#                     conv_layers=block_conv_layers,
#                     activation=activation,
#                     strides=strides,
#                     batch_norm=batch_norm,
#                     **common_params
#                 )
#             )
#             self.decoding_layers.append(
#                 UnetDecodingLayer(
#                     kernel_size=kernel_size,
#                     upsampling_kernel_size=resampling_kernel_size,
#                     filters=int(filter_scaling**(i) * filters),
#                     conv_layers=block_conv_layers,
#                     activation=activation,
#                     bilinear=upsampling_interpolation,
#                     batch_norm=batch_norm,
#                     **common_params
#                 )
#             )
#             self.gated_recurrent_blocks.append(
#                     ConvGRUBlock(
#                         filters=int(filter_scaling**(i) * filters),
#                         kernel_size=gru_kernel_size,
#                         activation=activation
#                 )
#             )

#         self.decoding_layers = self.decoding_layers[::-1]

#         self.bottleneck_layer1 = tf.keras.layers.Conv2D(
#             filters=bottleneck_filters,
#             kernel_size=bottleneck_kernel_size,
#             activation=activation,
#             **common_params
#         )
#         self.bottleneck_layer2 = tf.keras.layers.Conv2D(
#             filters=bottleneck_filters,
#             kernel_size=bottleneck_kernel_size,
#             activation=activation,
#             **common_params
#         )
#         self.bottleneck_gru = ConvGRUBlock(
#             filters=bottleneck_filters,
#             kernel_size=bottleneck_kernel_size,
#             activation=activation
#         )

#         self.output_layer = tf.keras.layers.Conv2D(
#             filters=2,  # source and kappa
#             kernel_size=(1, 1),
#             activation="linear",
#             **common_params
#         )        
#         self.input_layer = tf.keras.layers.Conv2D(
#             filters=filters,
#             kernel_size=input_kernel_size,
#             activation=activation,
#             **common_params
#         )

# #     def __call__(self, source, kappa, source_grad, kappa_grad, states, lens_pred, lens_true):
# #         return self.call(source, kappa, source_grad, kappa_grad, states, lens_pred, lens_true)

# #     def call(self, source, kappa, source_grad, kappa_grad, states, lens_pred, lens_true):
# #         delta_xt = tf.concat([source, source_grad, kappa, kappa_grad, lens_pred, lens_true], axis=-1)
#     def __call__(self, source, kappa, source_grad, kappa_grad, states):
#         return self.call(source, kappa, source_grad, kappa_grad, states)

#     def call(self, source, kappa, source_grad, kappa_grad, states):
#         delta_xt = tf.concat([source, source_grad, kappa, kappa_grad], axis=-1)
#         delta_xt = self.input_layer(delta_xt)
#         skip_connections = []
#         new_states = []
#         for i in range(len(self.encoding_layers)):
#             c_i, delta_xt = self.encoding_layers[i](delta_xt)
#             c_i, new_state = self.gated_recurrent_blocks[i](c_i, states[i])  # Pass skip connections through GRUS and update states
#             skip_connections.append(c_i)
#             new_states.append(new_state)
#         skip_connections = skip_connections[::-1]
#         delta_xt = self.bottleneck_layer1(delta_xt)
#         delta_xt = self.bottleneck_layer2(delta_xt)
#         delta_xt, new_state = self.bottleneck_gru(delta_xt, states[-1])
#         new_states.append(new_state)
#         for i in range(len(self.decoding_layers)):
#             delta_xt = self.decoding_layers[i](delta_xt, skip_connections[i])
#         delta_xt = self.output_layer(delta_xt)
#         source_delta, kappa_delta = tf.split(delta_xt, 2, axis=-1)
#         new_source = source + source_delta
#         new_kappa = kappa + kappa_delta
#         return new_source, new_kappa, new_states

#     def init_hidden_states(self, input_pixels, batch_size, constant=0.):
#         hidden_states = []
#         for i in range(self._num_layers):
#             pixels = input_pixels // self._strides**(i)
#             filters = int(self._filter_scaling**(i) * self._init_filters)
#             hidden_states.append(
#                 constant * tf.ones(shape=[batch_size, pixels, pixels, 2*filters], dtype=DTYPE)
#             )
#         pixels = input_pixels // self._strides ** (self._num_layers)
#         hidden_states.append(
#             constant * tf.ones(shape=[batch_size, pixels, pixels, 2*self._bottleneck_filters], dtype=DTYPE)
#         )
#         return hidden_states



In [14]:
# import tensorflow as tf
# # from censai.models import SharedUnetModel
# from censai.definitions import logkappa_normalization, log_10, DTYPE, logit, lrelu4p
# from censai import PhysicalModel
# from censai.utils import nulltape

# # LAM = -0.10455389589556062
# # LAM = -0.5369885858903738
# # LAM = 0.42627448994956957 # applied with log_10
# # LAM = -1.8944241461145863
# LAM = -0.8
# def boxcox(x):
#     return (x**LAM - 1) / LAM

# def boxcox_inverse(x):
# #     return tf.math.exp(tf.math.log(G**(LAM-1) * LAM * x + 1) / LAM)
#     return tf.math.exp(tf.math.log(G * LAM * x + 1) / LAM)

# class RIMSharedUnet:
#     """
#     Architecture has only 1 Unet. Source and kappa information are stacked along channel dimension.

#     There are 2 intended structures:
#         1. Kappa has a larger shape than Source tensor:
#             1 - Use a half-strided convolution to upsample the output of the Unet
#             3 - Use bilinear interpolation to upsample
#         2. Kappa and Source have the same tensor shape -> Identity layer

#     In any case, we use the Source shape for the Unet
#     """
#     def __init__(
#             self,
#             physical_model: PhysicalModel,
#             unet: SharedUnetModel,
#             steps: int,
#             adam=True,
#             kappalog=True,
#             kappa_normalize=False,
#             source_link="relu",
#             beta_1=0.99,
#             beta_2=0.999,
#             epsilon=1e-16,
#             kappa_init=1e-1,
#             source_init=1e-3
#     ):
#         self.physical_model = physical_model
#         self.kappa_pixels = physical_model.kappa_pixels
#         self.source_pixels = physical_model.src_pixels
#         self.unet = unet
#         self.steps = steps
#         self.adam = adam
#         self.kappalog = kappalog
#         self._source_link_func = source_link
#         self.kappa_normalize = kappa_normalize
#         self.beta_1 = beta_1
#         self.beta_2 = beta_2
#         self.epsilon = epsilon
#         self._kappa_init = kappa_init
#         self._source_init = source_init
        
# #         self.kappa_inverse_link = tf.keras.layers.Lambda(lambda x: boxcox(log_10(x) + 3))
# #         self.kappa_link = tf.keras.layers.Lambda(lambda x: 10**(boxcox_inverse(x) - 3))

#         if self.kappalog:
#             if self.kappa_normalize:
#                 self.kappa_inverse_link = tf.keras.layers.Lambda(lambda x: logkappa_normalization(log_10(x), forward=True))
#                 self.kappa_link = tf.keras.layers.Lambda(lambda x: 10**(logkappa_normalization(x, forward=False)))
#             else:
#                 self.kappa_inverse_link = tf.keras.layers.Lambda(lambda x: log_10(x))
#                 self.kappa_link = tf.keras.layers.Lambda(lambda x: 10**x)
#         else:
#             self.kappa_link = tf.identity
#             self.kappa_inverse_link = tf.identity
        

#         if self._source_link_func == "exp":
#             self.source_inverse_link = tf.keras.layers.Lambda(lambda x: tf.math.log(x + 1e-6))
#             self.source_link = tf.keras.layers.Lambda(lambda x: tf.math.exp(x))
#         elif self._source_link_func == "identity":
#             self.source_inverse_link = tf.identity
#             self.source_link = tf.identity
#         elif self._source_link_func == "relu":
#             self.source_inverse_link = tf.identity
#             self.source_link = tf.nn.relu
#         elif self._source_link_func == "sigmoid":
#             self.source_inverse_link = logit
#             self.source_link = tf.nn.sigmoid
#         elif self._source_link_func == "lrelu4p":
#             self.source_inverse_link = tf.identity
#             self.source_link = lrelu4p
#         else:
#             raise NotImplementedError(f"{source_link} not in ['exp', 'identity', 'relu', 'lrelu4p', 'sigmoid']")

#     def initial_states(self, batch_size):
#         # Define initial guess in physical space, then apply inverse link function to bring them in prediction space
#         source_init = self.source_inverse_link(tf.ones(shape=(batch_size, self.source_pixels, self.source_pixels, 1)) * self._source_init)
#         kappa_init = self.kappa_inverse_link(tf.ones(shape=(batch_size, self.kappa_pixels, self.kappa_pixels, 1)) * self._kappa_init)
#         states = self.unet.init_hidden_states(self.source_pixels, batch_size)
#         return source_init, kappa_init, states

#     def grad_update(self, grad1, grad2, time_step):
#         if self.adam:
#             if time_step == 0:  # reset mean and variance for time t=-1
#                 self._grad_mean1 = tf.zeros_like(grad1)
#                 self._grad_var1 = tf.zeros_like(grad1)
#                 self._grad_mean2 = tf.zeros_like(grad2)
#                 self._grad_var2 = tf.zeros_like(grad2)
#             self._grad_mean1 = self. beta_1 * self._grad_mean1 + (1 - self.beta_1) * grad1
#             self._grad_var1  = self.beta_2 * self._grad_var1 + (1 - self.beta_2) * tf.square(grad1)
#             self._grad_mean2 = self. beta_1 * self._grad_mean2 + (1 - self.beta_1) * grad2
#             self._grad_var2  = self.beta_2 * self._grad_var2 + (1 - self.beta_2) * tf.square(grad2)
#             # for grad update, unbias the moments
#             m_hat1 = self._grad_mean1 / (1 - self.beta_1**(time_step + 1))
#             v_hat1 = self._grad_var1 / (1 - self.beta_2**(time_step + 1))
#             m_hat2 = self._grad_mean2 / (1 - self.beta_1**(time_step + 1))
#             v_hat2 = self._grad_var2 / (1 - self.beta_2**(time_step + 1))
#             return m_hat1 / (tf.sqrt(v_hat1) + self.epsilon), m_hat2 / (tf.sqrt(v_hat2) + self.epsilon)
#         else:
#             return grad1, grad2

# #     def time_step(self, source, kappa, source_grad, kappa_grad, states, lens_pred, lens_true):
# #         source, kappa, states = self.unet(source, kappa, source_grad, kappa_grad, states, lens_pred, lens_true)
# #         return source, kappa, states

#     def time_step(self, source, kappa, source_grad, kappa_grad, states):
#         source, kappa, states = self.unet(source, kappa, source_grad, kappa_grad, states)
#         return source, kappa, states


#     def __call__(self, lensed_image, outer_tape=nulltape):
#         return self.call(lensed_image, outer_tape)

#     def call(self, lensed_image, outer_tape=nulltape):
#         """
#         Used in training. Return linked kappa and source maps.
#         """
#         batch_size = lensed_image.shape[0]
#         source, kappa, states = self.initial_states(batch_size)
# #         with outer_tape.stop_recording():
# #             y_true = tf.image.resize(lensed_image, size=(128, 128), antialias=True)
#         source_series = tf.TensorArray(DTYPE, size=self.steps)  # equivalent to empty list and append, but using tensorflow
#         kappa_series = tf.TensorArray(DTYPE, size=self.steps)
#         chi_squared_series = tf.TensorArray(DTYPE, size=self.steps)
#         for current_step in range(self.steps):
#             with outer_tape.stop_recording():
#                 with tf.GradientTape() as g:
#                     g.watch(source)
#                     g.watch(kappa)
#                     y_pred = self.physical_model.forward(source, kappa)
# #                     y_pred /= tf.reduce_max(y_pred, keepdims=True)
# #                     lam = tf.reduce_sum(lensed_image * y_pred, axis=(1, 2, 3)) / tf.reduce_sum(lensed_image**2, axis=(1, 2, 3))[..., tf.newaxis, tf.newaxis, tf.newaxis]
#                     log_likelihood = 0.5 * tf.reduce_sum((y_pred - lensed_image) ** 2 / self.physical_model.noise_rms ** 2, axis=(1, 2, 3))
#                     cost = tf.reduce_mean(log_likelihood)
#                 source_grad, kappa_grad = g.gradient(cost, [source, kappa])
#                 source_grad, kappa_grad = self.grad_update(source_grad, kappa_grad, current_step)
# #                 y_pred = tf.image.resize(y_pred, size=(128, 128), antialias=True)
# #                 y_pred /= tf.reduce_max(y_pred, keepdims=True)
# #             source, kappa, states = self.time_step(source, kappa, source_grad, kappa_grad, states, y_pred, y_true)
#             source, kappa, states = self.time_step(source, kappa, source_grad, kappa_grad, states)
#             source_series = source_series.write(index=current_step, value=source)
#             kappa_series = kappa_series.write(index=current_step, value=kappa)
#             chi_squared_series = chi_squared_series.write(index=current_step, value=log_likelihood)
#         return source_series.stack(), kappa_series.stack(), chi_squared_series.stack()

#     def predict(self, lensed_image):
#         """
#         Used in inference. Return physical kappa and source maps.
#         """
#         batch_size = lensed_image.shape[0]
#         source, kappa, states = self.initial_states(batch_size)
# #         y_true = tf.image.resize(lensed_image, size=(128, 128), antialias=True)
        
#         source_series = tf.TensorArray(DTYPE, size=self.steps)  # equivalent to empty list and append, but using tensorflow
#         kappa_series = tf.TensorArray(DTYPE, size=self.steps)
#         chi_squared_series = tf.TensorArray(DTYPE, size=self.steps)
#         for current_step in range(self.steps):
#             with tf.GradientTape() as g:
#                 g.watch(source)
#                 g.watch(kappa)
#                 y_pred = self.physical_model.forward(source, kappa)
# #                 y_pred /= tf.reduce_max(y_pred, keepdims=True)
# #                 lam = tf.reduce_sum(lensed_image * y_pred, axis=(1, 2, 3)) / tf.reduce_sum(lensed_image**2, axis=(1, 2, 3))[..., tf.newaxis, tf.newaxis, tf.newaxis]
#                 log_likelihood = 0.5 * tf.reduce_sum((y_pred - lensed_image) ** 2 / self.physical_model.noise_rms ** 2, axis=(1, 2, 3))
#                 cost = tf.reduce_mean(log_likelihood)
#             source_grad, kappa_grad = g.gradient(cost, [source, kappa])
#             source_grad, kappa_grad = self.grad_update(source_grad, kappa_grad, current_step)
# #             y_pred = tf.image.resize(y_pred, size=(128, 128), antialias=True)  
# #             y_pred /= tf.reduce_max(y_pred, keepdims=True)

# #             source, kappa, states = self.time_step(source, kappa, source_grad, kappa_grad, states, y_pred, y_true)  
#             source, kappa, states = self.time_step(source, kappa, source_grad, kappa_grad, states)            
#             source_series = source_series.write(index=current_step, value=self.source_link(source))
#             kappa_series = kappa_series.write(index=current_step, value=self.kappa_link(kappa))
#             chi_squared_series = chi_squared_series.write(index=current_step, value=log_likelihood)
#         return source_series.stack(), kappa_series.stack(), chi_squared_series.stack()  # stack along 0-th dimension

#     def cost_function(self, lensed_image, source, kappa, outer_tape=nulltape, reduction=True):
#         """

#         Args:
#             lensed_image: Batch of lensed images
#             source: Batch of source images
#             kappa: Batch of kappa maps
#             reduction: Whether or not to reduce the batch dimension in computing the loss or not

#         Returns: The average loss over pixels, time steps and (if reduction=True) batch size.

#         """
#         source_series, kappa_series, chi_squared = self.call(lensed_image, outer_tape=outer_tape)
#         source_cost = tf.reduce_sum(tf.square(source_series - self.source_inverse_link(source)), axis=0) / self.steps
#         kappa_cost = tf.reduce_sum(tf.square(kappa_series - self.kappa_inverse_link(kappa)), axis=0) / self.steps
#         chi = tf.reduce_sum(chi_squared, axis=0) / self.steps

#         if reduction:
#             return tf.reduce_mean(source_cost) + tf.reduce_mean(kappa_cost), tf.reduce_mean(chi)
#         else:
#             return tf.reduce_mean(source_cost, axis=(1, 2, 3)) + tf.reduce_mean(kappa_cost, axis=(1, 2, 3)), chi



In [26]:
def main(args):
    files = []
    for dataset in args.datasets:
        files.extend(glob.glob(os.path.join(dataset, "*.tfrecords")))
    np.random.shuffle(files)
    # Read concurrently from multiple records
    files = tf.data.Dataset.from_tensor_slices(files)
    dataset = files.interleave(lambda x: tf.data.TFRecordDataset(x, compression_type=args.compression_type),
                               block_length=args.block_length, num_parallel_calls=tf.data.AUTOTUNE)
    # Read off global parameters from first example in dataset
    for physical_params in dataset.map(decode_physical_model_info):
        break
    # preprocessing
    dataset = dataset.map(decode_train).map(preprocess)
    if args.cache_file is not None:
        dataset = dataset.cache(args.cache_file)
    train_dataset = dataset.shuffle(buffer_size=args.buffer_size).take(math.floor(args.train_split * args.total_items))\
        .batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = dataset.skip(math.floor(args.train_split * args.total_items))\
        .take(math.ceil((1 - args.train_split) * args.total_items)).batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    # train_dataset = STRATEGY.experimental_distribute_dataset(train_dataset)
    # val_dataset = STRATEGY.experimental_distribute_dataset(val_dataset)
    if args.raytracer is not None:
        with open(os.path.join(args.raytracer, "ray_tracer_hparams.json"), "r") as f:
            raytracer_hparams = json.load(f)
    # with STRATEGY.scope():  # Replicate ops accross gpus
    if args.raytracer is not None:
        raytracer = RayTracer(**raytracer_hparams)
        # load last checkpoint in the checkpoint directory
        checkpoint = tf.train.Checkpoint(net=raytracer)
        manager = tf.train.CheckpointManager(checkpoint, directory=args.raytracer, max_to_keep=3)
        checkpoint.restore(manager.latest_checkpoint).expect_partial()
    else:
        raytracer = None
    phys = PhysicalModel(
        pixels=physical_params["pixels"].numpy(),
        kappa_pixels=physical_params["kappa pixels"].numpy(),
        src_pixels=physical_params["src pixels"].numpy(),
        image_fov=physical_params["image fov"].numpy(),
        kappa_fov=physical_params["kappa fov"].numpy(),
        src_fov=physical_params["source fov"].numpy(),
        method=args.forward_method,
        noise_rms=physical_params["noise rms"].numpy(),
        raytracer=raytracer,
        psf_sigma=physical_params["psf sigma"].numpy()
    )

    unet = SharedUnetModel(
        filters=args.filters,
        filter_scaling=args.filter_scaling,
        kernel_size=args.kernel_size,
        layers=args.layers,
        block_conv_layers=args.block_conv_layers,
        strides=args.strides,
        bottleneck_kernel_size=args.bottleneck_kernel_size,
        bottleneck_filters=args.bottleneck_filters,
        resampling_kernel_size=args.resampling_kernel_size,
        input_kernel_size=args.input_kernel_size,
        gru_kernel_size=args.gru_kernel_size,
        upsampling_interpolation=args.upsampling_interpolation,
        kernel_l2_amp=args.kernel_l2_amp,
        bias_l2_amp=args.bias_l2_amp,
        kernel_l1_amp=args.kernel_l1_amp,
        bias_l1_amp=args.bias_l1_amp,
        activation=args.activation,
        alpha=args.alpha,
        initializer=args.initializer,
        batch_norm=args.batch_norm,
        dropout_rate=args.dropout_rate
    )
    rim = RIMSharedUnet(
        physical_model=phys,
        unet=unet,
        steps=args.steps,
        adam=args.adam,
        kappalog=args.kappalog,
        source_link=args.source_link,
        kappa_normalize=args.kappa_normalize,
        kappa_init=args.kappa_init,
        source_init=args.source_init
    )
    learning_rate_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=args.initial_learning_rate,
        decay_rate=args.decay_rate,
        decay_steps=args.decay_steps,
        staircase=args.staircase
    )
    optim = tf.keras.optimizers.deserialize(
        {
            "class_name": args.optimizer,
            'config': {"learning_rate": learning_rate_schedule}
        }
    )
    # weights for time steps in the loss function
    if args.time_weights == "uniform":
        wt = tf.ones(shape=(args.steps), dtype=DTYPE) / args.steps
    elif args.time_weights == "linear":
        wt = 2 * (tf.range(args.steps, dtype=DTYPE) + 1) / args.steps / (args.steps + 1)
    elif args.time_weights == "quadratic":
        wt = 6 * (tf.range(args.steps, dtype=DTYPE) + 1)**2 / args.steps / (args.steps + 1) / (2 * args.steps + 1)
    else:
        raise ValueError("time_weights must be in ['uniform', 'linear', 'quadratic']")
    wt = wt[..., tf.newaxis]  # [steps, batch]
    # ==== Take care of where to write logs and stuff =================================================================
    if args.model_id.lower() != "none":
        logname = args.model_id + "_" + args.logname if args.logname is not None else args.model_id
        model_id = args.model_id
    elif args.logname is not None:
        logname = args.logname
        model_id = logname
    else:
        logname = args.logname_prefixe + "_" + datetime.now().strftime("%y-%m-%d_%H-%M-%S")
        model_id = logname
    if args.logdir.lower() != "none":
        logdir = os.path.join(args.logdir, logname)
        if not os.path.isdir(logdir):
            os.mkdir(logdir)
        writer = tf.summary.create_file_writer(logdir)
    else:
        writer = nullwriter()
    # ===== Make sure directory and checkpoint manager are created to save model ===================================
    if args.model_dir.lower() != "none":
        checkpoints_dir = os.path.join(args.model_dir, logname)
        old_checkpoints_dir = os.path.join(args.model_dir, model_id)  # in case they differ we load model from a different directory
        if not os.path.isdir(checkpoints_dir):
            os.mkdir(checkpoints_dir)
            with open(os.path.join(checkpoints_dir, "script_params.json"), "w") as f:
                json.dump(vars(args), f, indent=4)
            with open(os.path.join(checkpoints_dir, "unet_hparams.json"), "w") as f:
                hparams_dict = {key: vars(args)[key] for key in UNET_MODEL_HPARAMS}
                json.dump(hparams_dict, f, indent=4)
            with open(os.path.join(checkpoints_dir, "rim_hparams.json"), "w") as f:
                hparams_dict = {key: vars(args)[key] for key in RIM_HPARAMS}
                json.dump(hparams_dict, f, indent=4)
        ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optim, net=rim.unet)
        checkpoint_manager = tf.train.CheckpointManager(ckpt, old_checkpoints_dir, max_to_keep=args.max_to_keep)
        save_checkpoint = True
        # ======= Load model if model_id is provided ===============================================================
        if args.model_id.lower() != "none":
            checkpoint_manager.checkpoint.restore(checkpoint_manager.latest_checkpoint)
        if old_checkpoints_dir != checkpoints_dir:  # save progress in another directory.
            checkpoint_manager = tf.train.CheckpointManager(ckpt, checkpoints_dir, max_to_keep=args.max_to_keep)
    else:
        save_checkpoint = False
    # =================================================================================================================

    def train_step(X, source, kappa):
        with tf.GradientTape() as tape:
            tape.watch(rim.unet.trainable_variables)
            source_series, kappa_series, chi_squared = rim.call(X, outer_tape=tape)
            # mean over image residuals
            source_cost = tf.reduce_mean(tf.square(source_series - rim.source_inverse_link(source)), axis=(2, 3, 4))
            kappa_cost = tf.reduce_mean(tf.square(kappa_series - rim.kappa_inverse_link(kappa)), axis=(2, 3, 4))
            # weighted mean over time steps
            source_cost = tf.reduce_sum(wt * source_cost, axis=0)
            kappa_cost = tf.reduce_sum(wt * kappa_cost, axis=0)
            # final cost is mean over global batch size
            cost = tf.reduce_sum(kappa_cost + source_cost) / args.batch_size
        gradient = tape.gradient(cost, rim.unet.trainable_variables)
        if args.clipping:
            gradient = [tf.clip_by_value(grad, -10, 10) for grad in gradient]
        optim.apply_gradients(zip(gradient, rim.unet.trainable_variables))
        chi_squared = tf.reduce_sum(chi_squared[-1]) / args.batch_size # take chi squared of converged prediction
        source_cost = tf.reduce_sum(source_cost) / args.batch_size
        kappa_cost = tf.reduce_sum(kappa_cost) / args.batch_size
        return cost, chi_squared, source_cost, kappa_cost

    # @tf.function
    # def distributed_train_step(dist_inputs):
    #     per_replica_losses, per_replica_chi_squared, per_replica_source_cost, per_replica_kappa_cost = STRATEGY.run(train_step, args=(dist_inputs,))
    #     # Replica losses are aggregated by summing them
    #     global_loss = STRATEGY.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)
    #     global_chi_squared = STRATEGY.reduce(tf.distribute.ReduceOp.SUM, per_replica_chi_squared, axis=None)
    #     global_source_cost = STRATEGY.reduce(tf.distribute.ReduceOp.SUM, per_replica_source_cost, axis=None)
    #     global_kappa_cost = STRATEGY.reduce(tf.distribute.ReduceOp.SUM, per_replica_kappa_cost, axis=None)
    #     return global_loss, global_chi_squared, global_source_cost, global_kappa_cost

    def test_step(X, source, kappa):
        source_series, kappa_series, chi_squared = rim.call(X)
        # mean over image residuals
        source_cost = tf.reduce_mean(tf.square(source_series - rim.source_inverse_link(source)), axis=(2, 3, 4))
        kappa_cost = tf.reduce_mean(tf.square(kappa_series - rim.kappa_inverse_link(kappa)), axis=(2, 3, 4))
        # weighted mean over time steps
        source_cost = tf.reduce_sum(wt * source_cost, axis=0)
        kappa_cost = tf.reduce_sum(wt * kappa_cost, axis=0)
        # final cost is mean over global batch size
        cost = tf.reduce_sum(kappa_cost + source_cost) / args.batch_size
        chi_squared = tf.reduce_sum(chi_squared[-1]) / args.batch_size
        source_cost = tf.reduce_sum(source_cost) / args.batch_size
        kappa_cost = tf.reduce_sum(kappa_cost) / args.batch_size
        return cost, chi_squared, source_cost, kappa_cost

    # @tf.function
    # def distributed_test_step(dist_inputs):
    #     per_replica_losses, per_replica_chi_squared, per_replica_source_cost, per_replica_kappa_cost = STRATEGY.run(test_step, args=(dist_inputs,))
    #     # Replica losses are aggregated by summing them
    #     global_loss = STRATEGY.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)
    #     global_chi_squared = STRATEGY.reduce(tf.distribute.ReduceOp.SUM, per_replica_chi_squared, axis=None)
    #     global_source_cost = STRATEGY.reduce(tf.distribute.ReduceOp.SUM, per_replica_source_cost, axis=None)
    #     global_kappa_cost = STRATEGY.reduce(tf.distribute.ReduceOp.SUM, per_replica_kappa_cost, axis=None)
    #     return global_loss, global_chi_squared, global_source_cost, global_kappa_cost

    # ====== Training loop ============================================================================================
    epoch_loss = tf.metrics.Mean()
    time_per_step = tf.metrics.Mean()
    val_loss = tf.metrics.Mean()
    epoch_chi_squared = tf.metrics.Mean()
    epoch_source_loss = tf.metrics.Mean()
    epoch_kappa_loss = tf.metrics.Mean()
    val_chi_squared = tf.metrics.Mean()
    val_source_loss = tf.metrics.Mean()
    val_kappa_loss = tf.metrics.Mean()
    history = {  # recorded at the end of an epoch only
        "train_cost": [],
        "train_chi_squared": [],
        "train_source_cost": [],
        "train_kappa_cost": [],
        "val_cost": [],
        "val_chi_squared": [],
        "val_source_cost": [],
        "val_kappa_cost": [],
        "learning_rate": [],
        "time_per_step": [],
        "step": [],
        "wall_time": []
    }
    best_loss = np.inf
    patience = args.patience
    step = 0
    global_start = time.time()
    estimated_time_for_epoch = 0
    out_of_time = False
    lastest_checkpoint = 1
    for epoch in range(args.epochs):
        if (time.time() - global_start) > args.max_time*3600 - estimated_time_for_epoch:
            break
        epoch_start = time.time()
        epoch_loss.reset_states()
        epoch_chi_squared.reset_states()
        epoch_source_loss.reset_states()
        epoch_kappa_loss.reset_states()
        time_per_step.reset_states()
        with writer.as_default():
            for batch, (X, source, kappa) in enumerate(train_dataset):
                start = time.time()
                cost, chi_squared, source_cost, kappa_cost = train_step(X, source, kappa)
        # ========== Summary and logs ==================================================================================
                _time = time.time() - start
                time_per_step.update_state([_time])
                epoch_loss.update_state([cost])
                epoch_chi_squared.update_state([chi_squared])
                epoch_source_loss.update_state([source_cost])
                epoch_kappa_loss.update_state([kappa_cost])
                step += 1
            # last batch we make a summary of residuals
            if args.n_residuals > 0:
                source_pred, kappa_pred, chi_squared = rim.predict(X)
                lens_pred = phys.forward(source_pred[-1], kappa_pred[-1])
                lam = phys.lagrange_multiplier(y_true=X, y_pred=lens_pred)
            for res_idx in range(min(args.n_residuals, args.batch_size)):
                try:
                    tf.summary.image(f"Residuals {res_idx}",
                                     plot_to_image(
                                         residual_plot(
                                             lam[res_idx]*X[res_idx],
                                             source[res_idx],
                                             kappa[res_idx],
                                             lens_pred[res_idx],
                                             source_pred[-1][res_idx],
                                             kappa_pred[-1][res_idx],
                                             chi_squared[-1][res_idx]
                                         )), step=step)
                except ValueError:
                    continue

            # ========== Validation set ===================
            val_loss.reset_states()
            val_chi_squared.reset_states()
            val_source_loss.reset_states()
            val_kappa_loss.reset_states()
            for X, source, kappa in val_dataset:
                cost, chi_squared, source_cost, kappa_cost = test_step(X, source, kappa)
                val_loss.update_state([cost])
                val_chi_squared.update_state([chi_squared])
                val_source_loss.update_state([source_cost])
                val_kappa_loss.update_state([kappa_cost])

            if args.n_residuals > 0 and math.ceil((1 - args.train_split) * args.total_items) > 0:  # validation set not empty set not empty
                source_pred, kappa_pred, chi_squared = rim.predict(X)
                lens_pred = phys.forward(source_pred[-1], kappa_pred[-1])
                lam = phys.lagrange_multiplier(y_true=X, y_pred=lens_pred)
            for res_idx in range(min(args.n_residuals, args.batch_size, math.ceil((1 - args.train_split) * args.total_items))):
                try:
                    tf.summary.image(f"Val Residuals {res_idx}",
                                     plot_to_image(
                                         residual_plot(
                                             lam[res_idx]*X[res_idx],  # rescale intensity like it is done in the likelihood
                                             source[res_idx],
                                             kappa[res_idx],
                                             lens_pred[res_idx],
                                             source_pred[-1][res_idx],
                                             kappa_pred[-1][res_idx],
                                             chi_squared[-1][res_idx]
                                         )), step=step)
                except ValueError:
                    continue
            val_cost = val_loss.result().numpy()
            train_cost = epoch_loss.result().numpy()
            val_chi_sq = val_chi_squared.result().numpy()
            train_chi_sq = epoch_chi_squared.result().numpy()
            val_kappa_cost = val_kappa_loss.result().numpy()
            train_kappa_cost = epoch_kappa_loss.result().numpy()
            val_source_cost = val_source_loss.result().numpy()
            train_source_cost = epoch_source_loss.result().numpy()
            tf.summary.scalar("Time per step", time_per_step.result(), step=step)
            tf.summary.scalar("Chi Squared", train_chi_sq, step=step)
            tf.summary.scalar("Kappa cost", train_kappa_cost, step=step)
            tf.summary.scalar("Val Kappa cost", val_kappa_cost, step=step)
            tf.summary.scalar("Source cost", train_source_cost, step=step)
            tf.summary.scalar("Val Source cost", val_source_cost, step=step)
            tf.summary.scalar("MSE", train_cost, step=step)
            tf.summary.scalar("Val MSE", val_cost, step=step)
            tf.summary.scalar("Learning Rate", optim.lr(step), step=step)
            tf.summary.scalar("Val Chi Squared", val_chi_sq, step=step)
        print(f"epoch {epoch} | train loss {train_cost:.3e} | val loss {val_cost:.3e} "
              f"| lr {optim.lr(step).numpy():.2e} | time per step {time_per_step.result().numpy():.2e} s"
              f"| kappa cost {train_kappa_cost:.2e} | source cost {train_source_cost:.2e} | chi sq {train_chi_sq:.2e}")
        history["train_cost"].append(train_cost)
        history["val_cost"].append(val_cost)
        history["learning_rate"].append(optim.lr(step).numpy())
        history["train_chi_squared"].append(train_chi_sq)
        history["val_chi_squared"].append(val_chi_sq)
        history["time_per_step"].append(time_per_step.result().numpy())
        history["train_kappa_cost"].append(train_kappa_cost)
        history["train_source_cost"].append(train_source_cost)
        history["val_kappa_cost"].append(val_kappa_cost)
        history["val_source_cost"].append(val_source_cost)
        history["step"].append(step)
        history["wall_time"].append(time.time() - global_start)

        cost = train_cost if args.track_train else val_cost
        if np.isnan(cost):
            print("Training broke the Universe")
            break
        if cost < (1 - args.tolerance) * best_loss:
            best_loss = cost
            patience = args.patience
        else:
            patience -= 1
        if (time.time() - global_start) > args.max_time * 3600:
            out_of_time = True
        if save_checkpoint:
            checkpoint_manager.checkpoint.step.assign_add(1) # a bit of a hack
            if epoch % args.checkpoints == 0 or patience == 0 or epoch == args.epochs - 1 or out_of_time:
                with open(os.path.join(checkpoints_dir, "score_sheet.txt"), mode="a") as f:
                    np.savetxt(f, np.array([[lastest_checkpoint, cost]]))
                lastest_checkpoint += 1
                checkpoint_manager.save()
                print("Saved checkpoint for step {}: {}".format(int(checkpoint_manager.checkpoint.step), checkpoint_manager.latest_checkpoint))
        if patience == 0:
            print("Reached patience")
            break
        if out_of_time:
            break
        if epoch > 0:  # First epoch is always very slow and not a good estimate of an epoch time.
            estimated_time_for_epoch = time.time() - epoch_start
    print(f"Finished training after {(time.time() - global_start)/3600:.3f} hours.")
    return rim, phys, train_dataset, val_dataset



In [27]:
from argparse import ArgumentParser
import json
parser = ArgumentParser()

parser.add_argument("--model_id",               default="None",                 help="Start from this model id checkpoint. None means start from scratch")
parser.add_argument("--datasets",               required=True,  nargs="+",      help="Path to directories that contains tfrecords of dataset. Can be multiple inputs (space separated)")
parser.add_argument("--compression_type",       default=None,                   help="Compression type used to write data. Default assumes no compression.")

# RIM hyperparameters
parser.add_argument("--steps",              default=16,     type=int,       help="Number of time steps of RIM")
parser.add_argument("--adam",               action="store_true",            help="ADAM update for the log-likelihood gradient.")
parser.add_argument("--kappalog",           action="store_true")
parser.add_argument("--kappa_normalize",    action="store_true")
parser.add_argument("--source_link",        default="identity",             help="One of 'exp', 'source', 'relu' or 'identity' (default).")
parser.add_argument("--kappa_init",         default=1e-1,   type=float,     help="Initial value of kappa for RIM")
parser.add_argument("--source_init",        default=1e-3,   type=float,     help="Initial value of source for RIM")

# Shared Unet params
parser.add_argument("--filters",                                    default=32,     type=int)
parser.add_argument("--filter_scaling",                             default=1,      type=float)
parser.add_argument("--kernel_size",                                default=3,      type=int)
parser.add_argument("--layers",                                     default=2,      type=int)
parser.add_argument("--block_conv_layers",                          default=2,      type=int)
parser.add_argument("--strides",                                    default=2,      type=int)
parser.add_argument("--bottleneck_kernel_size",                     default=None,   type=int)
parser.add_argument("--bottleneck_filters",                         default=None,   type=int)
parser.add_argument("--resampling_kernel_size",                     default=None,   type=int)
parser.add_argument("--input_kernel_size",                          default=11,     type=int)
parser.add_argument("--gru_kernel_size",                            default=None,   type=int)
parser.add_argument("--upsampling_interpolation",                   action="store_true")
parser.add_argument("--batch_norm",                                 action="store_true")
parser.add_argument("--dropout_rate",                               default=None,   type=float)
parser.add_argument("--kernel_l2_amp",                              default=0,      type=float)
parser.add_argument("--bias_l2_amp",                                default=0,      type=float)
parser.add_argument("--kernel_l1_amp",                              default=0,      type=float)
parser.add_argument("--bias_l1_amp",                                default=0,      type=float)
parser.add_argument("--activation",                                 default="leaky_relu")
parser.add_argument("--alpha",                                      default=0.1,    type=float)
parser.add_argument("--initializer",                                default="glorot_normal")
parser.add_argument("--gru_architecture",                           default="concat",   help="'concat': architecture of Laurence. 'plus': original RNN architecture")


# Physical model hyperparameter
parser.add_argument("--forward_method",         default="conv2d",               help="One of ['conv2d', 'fft', 'unet']. If the option 'unet' is chosen, the parameter "
                                                                                     "'--raytracer' must be provided and point to model checkpoint directory.")
parser.add_argument("--raytracer",              default=None,                   help="Path to raytracer checkpoint dir if method 'unet' is used.")

# Training set params
parser.add_argument("-b", "--batch_size",       default=1,      type=int,       help="Number of images in a batch. ")
parser.add_argument("--train_split",            default=0.8,    type=float,     help="Fraction of the training set.")
parser.add_argument("--total_items",            required=True,  type=int,       help="Total images in an epoch.")
# ... for tfrecord dataset
parser.add_argument("--cache_file",             default=None,                   help="Path to cache file, useful when training on server. Use ${SLURM_TMPDIR}/cache")
parser.add_argument("--block_length",           default=1,      type=int,       help="Number of example to read from each files at a given moment.")
parser.add_argument("--buffer_size",            default=1000,   type=int,       help="Buffer size for shuffling at each epoch.")

# Optimization params
parser.add_argument("-e", "--epochs",           default=10,     type=int,       help="Number of epochs for training.")
parser.add_argument("--optimizer",              default="Adam",                 help="Class name of the optimizer (e.g. 'Adam' or 'Adamax')")
parser.add_argument("--initial_learning_rate",  default=1e-3,   type=float,     help="Initial learning rate.")
parser.add_argument("--decay_rate",             default=1.,     type=float,     help="Exponential decay rate of learning rate (1=no decay).")
parser.add_argument("--decay_steps",            default=1000,   type=int,       help="Decay steps of exponential decay of the learning rate.")
parser.add_argument("--staircase",              action="store_true",            help="Learning rate schedule only change after decay steps if enabled.")
parser.add_argument("--clipping",               action="store_true",            help="Clip backprop gradients between -10 and 10.")
parser.add_argument("--patience",               default=np.inf, type=int,       help="Number of step at which training is stopped if no improvement is recorder.")
parser.add_argument("--tolerance",              default=0,      type=float,     help="Current score <= (1 - tolerance) * best score => reset patience, else reduce patience.")
parser.add_argument("--track_train",            action="store_true",            help="Track training metric instead of validation metric, in case we want to overfit")
parser.add_argument("--max_time",               default=np.inf, type=float,     help="Time allowed for the training, in hours.")
parser.add_argument("--time_weights",           default="uniform",              help="uniform: w_t=1 for all t, linear: w_t~t, quadratic: w_t~t^2")

# logs
parser.add_argument("--logdir",                  default="None",                help="Path of logs directory. Default if None, no logs recorded.")
parser.add_argument("--logname",                 default=None,                  help="Overwrite name of the log with this argument")
parser.add_argument("--logname_prefixe",         default="RIMUnet512",          help="If name of the log is not provided, this prefix is prepended to the date")
parser.add_argument("--model_dir",               default="None",                help="Path to the directory where to save models checkpoints.")
parser.add_argument("--checkpoints",             default=10,    type=int,       help="Save a checkpoint of the models each {%} iteration.")
parser.add_argument("--max_to_keep",             default=3,     type=int,       help="Max model checkpoint to keep.")
parser.add_argument("--n_residuals",             default=1,     type=int,       help="Number of residual plots to save. Add overhead at the end of an epoch only.")

# Reproducibility params
parser.add_argument("--seed",                   default=None,   type=int,       help="Random seed for numpy and tensorflow.")
parser.add_argument("--json_override",          default=None,    nargs="+",     help="A json filepath that will override every command line parameters. "
                                                                             "Useful for reproducibility")

_StoreAction(option_strings=['--json_override'], dest='json_override', nargs='+', const=None, default=None, type=None, choices=None, help='A json filepath that will override every command line parameters. Useful for reproducibility', metavar=None)

In [ ]:
import sys
sys.argv = ['']
args = parser.parse_args(
  f"--datasets {os.getenv('CENSAI_PATH')}/data/lenses512_hk128_TNG100_10k_verydiffuse "\
  f"--compression_type=GZIP "\
  f"--forward_method=fft "\
  f"--optimizer=ADAMAX "\
  f"--epochs=200 "\
  f"--max_time=1 "\
  f"--initial_learning_rate=5e-4 "\
  f"--decay_rate=0.5 "\
  f"--decay_steps=10 "\
  f"--staircase "\
  f"--clipping "\
  f"--patience=20 "\
#   f"--tolerance=0.01 "\
  f"--batch_size=10 "\
  f"--train_split=1 "\
  f"--total_items=100 "\
  f"--block_length=1 "\
  f"--buffer_size=100 "\
  f"--steps=15 "\
  f"--time_weights=quadratic "\
  f"--adam "\
  f"--kappalog "\
  f"--source_link=lrelu4p "\
  f"--filters=8 "\
  f"--filter_scaling=2 "\
  f"--kernel_size=3 "\
  f"--input_kernel_size=3 "\
  f"--layers=3 "\
  f"--block_conv_layers=1 "\
#   f"--resampling_kernel_size=3 "\
#   f"--gru_kernel_size=3 "\
#   f"--batch_norm "\
#   f"--dropout=0.1 "\
#   f"--upsampling_interpolation "\
#   f"--kernel_l2_amp=1e-4 "\
#   f"--bias_l2_amp=1e-4 "\
  f"--kernel_l1_amp=1e-3 "\
  f"--bias_l1_amp=1e-3 "\
  f"--activation=tanh "\
#   f"--alpha=0.3 "\
  f"--cache_file={os.getenv('SLURM_TMPDIR')}/cache "\
  f"--logdir={os.getenv('HOME')}/scratch/Censai/logs "\
  f"--logname_prefixe=RIMSU512_k128_hTNG_interactive "\
  f"--track_train "\
  f"--model_dir={os.getenv('HOME')}/scratch/Censai/models "\
  f"--checkpoints=5 "\
  f"--max_to_keep=3 "\
  f"--seed=42 "\
  f"--n_residuals=0 ".split()
#   f"--json_override {os.getenv('CENSAI_PATH')}/models/{model_id}/unet_hparams.json {os.getenv('CENSAI_PATH')}/models/{model_id}/rim_hparams.json".split()
)
if args.json_override is not None:
    if isinstance(args.json_override, list):
        files = args.json_override
    else:
        files = [args.json_override,]
    for file in files:
        with open(file, "r") as f:
            json_override = json.load(f)
        args_dict = vars(args)
        args_dict.update(json_override)
cache_files = glob.glob(f"{os.getenv('SLURM_TMPDIR')}/cache*")
for cache in cache_files:
    os.remove(cache)
rim, phys, train_dataset, val_dataset = main(args)

In [ ]:
for batch, distributed_inputs in enumerate(train_dataset):
    for res_idx in range(args.batch_size):
        lens_true = distributed_inputs[0][res_idx]
        lens_true /= tf.reduce_max(lens_true, axis=(0, 1, 2), keepdims=True)
        source_true = distributed_inputs[1][res_idx]
        kappa_true = distributed_inputs[2][res_idx]
        source_pred, kappa_pred, chi_squared = rim.predict(lens_true[None])
        lens_pred = phys.forward(source_pred[-1], kappa_pred[-1])[0]
#         lam = tf.reduce_sum(lens_true * lens_pred) / tf.reduce_sum(lens_true**2)[..., tf.newaxis, tf.newaxis, tf.newaxis]
        fig = residual_plot(lens_true, source_true, kappa_true, lens_pred, source_pred[-1][0, ...], kappa_pred[-1][0, ...], chi_squared[-1][0])
        fig.suptitle(fr"{batch}: Time step {args.steps+1}, $\chi^2 = ${chi_squared[-1][0]:.2e}")
    break

In [ ]:
model_id = "RIMSU512_k128_NIE2nsvdO_007_TS10_F8_L4_IK7_NLrelu_al0.1_GAconcat_82_B10_lr0.0005_dr1.0_ds5000_TWquadratic_210923024919"
import sys
sys.argv = ['']
args = parser.parse_args(
  f"--model_id={os.getenv('CENSAI_PATH')}/models/{model_id} "\
  f"--datasets {os.getenv('CENSAI_PATH')}/data/lenses512_hk128_TNG100_10k_verydiffuse "\
  f"--compression_type=GZIP "\
  f"--forward_method=fft "\
  f"--optimizer=ADAM "\
  f"--epochs=200 "\
  f"--max_time=1 "\
  f"--initial_learning_rate=1e-3 "\
  f"--decay_rate=0.5 "\
  f"--decay_steps=100 "\
  f"--staircase "\
  f"--clipping "\
  f"--patience=20 "\
  f"--tolerance=0.01 "\
  f"--batch_size=10 "\
  f"--train_split=1 "\
  f"--total_items=1000 "\
  f"--block_length=1 "\
  f"--buffer_size=1000 "\
#   f"--steps=10 "\
#   f"--adam "\
#   f"--kappalog "\
#   f"--source_link=lrelu4p "\
#   f"--filters=16 "\
#   f"--filter_scaling=1.5 "\
#   f"--kernel_size=3 "\
#   f"--layers=5 "\
#   f"--block_conv_layers=2 "\
#   f"--resampling_kernel_size=3 "\
#   f"--gru_kernel_size=3 "\
#   f"--batch_norm "\
#   f"--dropout=0.1 "\
#   f"--upsampling_interpolation "\
#   f"--kernel_l2_amp=1e-4 "\
#   f"--bias_l2_amp=1e-4 "\
#   f"--kernel_l1_amp=1e-3 "\5e-3
#   f"--bias_l1_amp=1e-3 "\
#   f"--activation=leaky_relu "\
#   f"--alpha=0.3 "\
  f"--cache_file={os.getenv('SLURM_TMPDIR')}/cache "\
  f"--logdir={os.getenv('HOME')}/scratch/Censai/logs "\
  f"--logname=RIMSU512_k128_pretrained_hTNG "\
  f"--track_train "\
  f"--model_dir={os.getenv('HOME')}/scratch/Censai/models "\
  f"--checkpoints=5 "\
  f"--max_to_keep=3 "\
  f"--seed=42 "\
  f"--n_residuals=0 "\
  f"--json_override {os.getenv('CENSAI_PATH')}/models/{model_id}/unet_hparams.json {os.getenv('CENSAI_PATH')}/models/{model_id}/rim_hparams.json".split()

)
if args.json_override is not None:
    if isinstance(args.json_override, list):
        files = args.json_override
    else:
        files = [args.json_override,]
    for file in files:
        with open(file, "r") as f:
            json_override = json.load(f)
        args_dict = vars(args)
        args_dict.update(json_override)
cache_files = glob.glob(f"{os.getenv('SLURM_TMPDIR')}/cache*")
for cache in cache_files:
    os.remove(cache)
rim, phys, train_dataset, val_dataset = main(args)